In [23]:
import pandas as pd 
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

base = pd.read_csv('games.csv')
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


### Limpando a base

In [24]:
base.drop(columns=['Other_Sales','Global_Sales','Developer'], inplace=True)

In [25]:
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN


In [26]:
base.shape

(16719, 13)

In [27]:
base.dropna(axis=0, inplace=True)

In [28]:
base.shape

(6825, 13)

In [29]:
base = base.loc[base['NA_Sales'] > 1]
base.shape

(584, 13)

In [30]:
base = base.loc[base['EU_Sales'] > 1]
base.shape

(258, 13)

In [31]:
base['Name'].value_counts()

Grand Theft Auto V                 4
Assassin's Creed IV: Black Flag    3
Call of Duty: Ghosts               3
Battlefield 4                      3
The Elder Scrolls V: Skyrim        3
                                  ..
Tekken 2                           1
Destiny                            1
Cooking Mama                       1
Forza Motorsport 3                 1
The Sims: Bustin' Out              1
Name: Name, Length: 223, dtype: int64

In [32]:
nome_jogos = base.Name
nome_jogos

0                      Wii Sports
2                  Mario Kart Wii
3               Wii Sports Resort
6           New Super Mario Bros.
7                        Wii Play
                  ...            
568                Medal of Honor
573    Tom Clancy's Splinter Cell
591        Mario Strikers Charged
610                    Crazy Taxi
636         The Sims: Bustin' Out
Name: Name, Length: 258, dtype: object

In [33]:
base.drop('Name', axis =1, inplace= True)
base.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [34]:
previsores = base.iloc[:,[0,1,2,3,7,8,9,10,11]].values
previsores

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['Wii', 2007.0, 'Sports', ..., '8', 124.0, 'E10+'],
       ['PS2', 2001.0, 'Racing', ..., '7.9', 46.0, 'T'],
       ['PS2', 2003.0, 'Simulation', ..., '8.4', 42.0, 'T']], dtype=object)

In [35]:
venda_na = base.iloc[:,4].values
venda_eu = base.iloc[:,5].values
venda_jp = base.iloc[:,6].values

In [36]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
from sklearn.compose import ColumnTransformer

labelencoder = LabelEncoder()
mudar = [0,2,3,8]
for i in mudar:
    previsores[:,i] = labelencoder.fit_transform(previsores[:,i])
previsores

array([[10, 2006.0, 10, ..., '8', 322.0, 0],
       [10, 2008.0, 6, ..., '8.3', 709.0, 0],
       [10, 2009.0, 10, ..., '8', 192.0, 0],
       ...,
       [10, 2007.0, 10, ..., '8', 124.0, 1],
       [6, 2001.0, 6, ..., '7.9', 46.0, 3],
       [6, 2003.0, 9, ..., '8.4', 42.0, 3]], dtype=object)

In [37]:
labelencoder_previsores = LabelEncoder()
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), mudar)],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()
previsores

array([[  0. ,   0. ,   0. , ...,  51. ,   8. , 322. ],
       [  0. ,   0. ,   0. , ...,  73. ,   8.3, 709. ],
       [  0. ,   0. ,   0. , ...,  73. ,   8. , 192. ],
       ...,
       [  0. ,   0. ,   0. , ...,  47. ,   8. , 124. ],
       [  0. ,   0. ,   0. , ...,  15. ,   7.9,  46. ],
       [  0. ,   0. ,   0. , ...,  26. ,   8.4,  42. ]])

In [38]:
previsores.shape

(258, 61)

In [39]:
camada_entrada = Input(shape=(61,))
camada_oculta1 = Dense(units = 32, activation = 'sigmoid')(camada_entrada)
camada_oculta2 = Dense(units = 32, activation = 'sigmoid')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [40]:
regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, camada_saida3])
regressor.compile(optimizer = 'adam',
                  loss = 'mse')
regressor.fit(previsores, [venda_na, venda_eu, venda_jp],
              epochs = 5000, batch_size = 100)

Epoch 1/5000
3/3 [==============================] - 1s 3ms/step - loss: 46.1637 - dense_2_loss: 27.6884 - dense_3_loss: 16.7104 - dense_4_loss: 1.7649
Epoch 2/5000
3/3 [==============================] - 0s 2ms/step - loss: 44.4500 - dense_2_loss: 26.7709 - dense_3_loss: 15.9953 - dense_4_loss: 1.6838
Epoch 3/5000
3/3 [==============================] - 0s 3ms/step - loss: 42.8630 - dense_2_loss: 25.9158 - dense_3_loss: 15.3377 - dense_4_loss: 1.6095
Epoch 4/5000
3/3 [==============================] - 0s 3ms/step - loss: 41.3591 - dense_2_loss: 25.0926 - dense_3_loss: 14.7233 - dense_4_loss: 1.5432
Epoch 5/5000
3/3 [==============================] - 0s 3ms/step - loss: 39.8950 - dense_2_loss: 24.2857 - dense_3_loss: 14.1256 - dense_4_loss: 1.4838
Epoch 6/5000
3/3 [==============================] - 0s 2ms/step - loss: 38.4820 - dense_2_loss: 23.5099 - dense_3_loss: 13.5483 - dense_4_loss: 1.4238
Epoch 7/5000
3/3 [==============================] - 0s 2ms/step - loss: 37.1773 - dense_2_loss

In [41]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)

9/9 [==============================] - 0s 995us/step
